#  Consignes

## Description

Ouvrir le fichier ks-projects-201801.csv, il recense environ 100 000 projets KickStarter. Intégrer les données directement avec L'API Python dans une base de données Mongo. 

Il conviendra de bien spécifier manuellement l'ID du document. Pensez aussi à bien formatter le type des données pour profiter des méthodes implémentées par Mongo. L'ensemble de données n'est pas forcément nécessaire, c'est à vous de créer votre modèle de données.

## Questions

- 1) Récupérer les 5 projets ayant reçu le plus de promesse de dons.
- 2) Compter le nombre de projets ayant atteint leur but.
- 3) Compter le nombre de projets pour chaque catégorie.
- 4) Compter le nombre de projets français ayant été instanciés avant 2016.
- 5) Récupérer les projets américains ayant demandé plus de 200 000 dollars.
- 6) Compter le nombre de projet ayant "Sport" dans leur nom

In [1]:
import pandas as pd
import pymongo

/root/.local/share/virtualenvs/code-AFz48Hjb/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
client = pymongo.MongoClient('mongo')
database = client['exercices']
collection = database['kickstarter']

In [3]:
df_ks = pd.read_csv("./data/ks-projects-201801-sample.csv")
df_ks.head()

/root/.local/share/virtualenvs/code-AFz48Hjb/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3172: DtypeWarning: Columns (6,8,10,12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real
0,872782264,"Scott Cooper's Solo CD ""A Leg Trick"" (Canceled)",Rock,Music,USD,2011-09-16,2000,2011-08-17 06:31:31,1145,canceled,24,US,1145,1145.000000
1,1326492673,Ohceola jewelry,Fashion,Fashion,USD,2012-08-22,18000,2012-07-23 20:46:48,1851,failed,28,US,1851,1851.000000
2,1688410639,Sluff Off & Harald: Two latest EGGs are Classi...,Tabletop Games,Games,USD,2016-07-19,2000,2016-07-01 21:55:54,7534,successful,254,US,3796,7534.000000
3,156812982,SketchPlanner: Create and Plan- all in one bea...,Art Books,Publishing,USD,2017-09-27,13000,2017-08-28 15:47:02,16298,successful,367,US,2670,16298.000000
4,1835968190,Proven sales with custom motorcycle accessories,Sculpture,Art,CAD,2016-02-24,5000,2016-01-25 17:37:10,1,failed,1,CA,0.708148,0.738225


Ce warning intervient lorsque pandas n'arrive pas à inférer le type de données. Il est sympa il précise les colones 6,8,10,12. 

In [4]:
df_ks.columns[[6,8,10,12]]

Index(['goal', 'pledged', 'backers', 'usd pledged'], dtype='object')

## Question 0

### Netoyer les données

In [5]:
df_ks.dtypes

ID                    int64
name                 object
category             object
main_category        object
currency             object
deadline             object
goal                 object
launched             object
pledged              object
state                object
backers              object
country              object
usd pledged          object
usd_pledged_real    float64
dtype: object

In [6]:
#passae au type float pour tout les colonnes qui devrai l'etre
colonne = ['goal', 'pledged', 'backers', 'usd pledged']
for i in colonne:
    df_ks[i] = pd.to_numeric(df_ks[i], errors='coerce')

In [7]:
#on voit bien le changement de type par rapport au type precedeùent obtenue
df_ks.dtypes

ID                    int64
name                 object
category             object
main_category        object
currency             object
deadline             object
goal                float64
launched             object
pledged             float64
state                object
backers             float64
country              object
usd pledged         float64
usd_pledged_real    float64
dtype: object

In [8]:
#on retire tout les valeur 'Na' et tout les duplicates de df_ks
df_ks = df_ks.dropna().drop_duplicates()

In [9]:
print(df_ks.dtypes)
print(df_ks.shape)

ID                    int64
name                 object
category             object
main_category        object
currency             object
deadline             object
goal                float64
launched             object
pledged             float64
state                object
backers             float64
country              object
usd pledged         float64
usd_pledged_real    float64
dtype: object
(148515, 14)


### Importer les données

In [11]:
#lors de l'execution de ce notebook assurons nous que la collection 'collection' soit bien vide
collection.delete_many({})

In [12]:
#creation d'une chaine de caractere 
db_ks = df_ks.to_json(orient = 'records')

In [13]:
from collections import Counter

Counter([type(data.get("pledged")) for data in json.loads(db_ks)])

NameError: name 'json' is not defined

In [14]:
from pathlib import Path
import json
Path("./ks.json").write_text(json.dumps(json.loads(db_ks), indent=4))

68750808

In [15]:
#remplissage de a collection a partir du csv precedent
collection.insert_many(json.loads(db_ks))

## Question 1  

In [16]:
#rename de la collection
collection_ks = collection

In [17]:
#pour voir a quoi ressemble un document de la collection
collection_ks.find_one()

{'_id': ObjectId('61b1dd1c90033986b96427a8'),
 'ID': 872782264,
 'name': 'Scott Cooper\'s Solo CD "A Leg Trick" (Canceled)',
 'category': 'Rock',
 'main_category': 'Music',
 'currency': 'USD',
 'deadline': '2011-09-16',
 'goal': 2000.0,
 'launched': '2011-08-17 06:31:31',
 'pledged': 1145.0,
 'state': 'canceled',
 'backers': 24.0,
 'country': 'US',
 'usd pledged': 1145.0,
 'usd_pledged_real': 1145.0}

In [18]:
cur = collection_ks.find()[:2]
list(cur)

[{'_id': ObjectId('61b1dd1c90033986b96427a8'),
  'ID': 872782264,
  'name': 'Scott Cooper\'s Solo CD "A Leg Trick" (Canceled)',
  'category': 'Rock',
  'main_category': 'Music',
  'currency': 'USD',
  'deadline': '2011-09-16',
  'goal': 2000.0,
  'launched': '2011-08-17 06:31:31',
  'pledged': 1145.0,
  'state': 'canceled',
  'backers': 24.0,
  'country': 'US',
  'usd pledged': 1145.0,
  'usd_pledged_real': 1145.0},
 {'_id': ObjectId('61b1dd1c90033986b96427a9'),
  'ID': 1326492673,
  'name': 'Ohceola jewelry',
  'category': 'Fashion',
  'main_category': 'Fashion',
  'currency': 'USD',
  'deadline': '2012-08-22',
  'goal': 18000.0,
  'launched': '2012-07-23 20:46:48',
  'pledged': 1851.0,
  'state': 'failed',
  'backers': 28.0,
  'country': 'US',
  'usd pledged': 1851.0,
  'usd_pledged_real': 1851.0}]

In [19]:
#recherche des 5 projets aillant recu le plus de promesse de dont
cur = collection_ks.find({},{"pledged":1, "name":1}).sort([("pledged", -1)])[:5]
list(cur)

[{'_id': ObjectId('61b1dd1d90033986b9654fc5'),
  'name': "COOLEST COOLER: 21st Century Cooler that's Actually Cooler",
  'pledged': 13285226.36},
 {'_id': ObjectId('61b1dd1d90033986b965a9ef'),
  'name': 'Pebble 2, Time 2 + All-New Pebble Core',
  'pledged': 12779843.49},
 {'_id': ObjectId('61b1dd1d90033986b9655bdd'),
  'name': 'Expect the Unexpected. digiFilmï¿½ Camera by YASHICA',
  'pledged': 10035296.0},
 {'_id': ObjectId('61b1dd1d90033986b965de86'),
  'name': 'OUYA: A New Kind of Video Game Console',
  'pledged': 8596474.58},
 {'_id': ObjectId('61b1dd1d90033986b9665508'),
  'name': 'The Everyday Backpack, Tote, and Sling',
  'pledged': 6565782.5}]

In [20]:
cur = collection_ks.find({"name" : 'The Girl in The Mountain - Film'})
next(cur)

{'_id': ObjectId('61b1dd1d90033986b9659943'),
 'ID': 796104705,
 'name': 'The Girl in The Mountain - Film',
 'category': 'Science Fiction',
 'main_category': 'Film & Video',
 'currency': 'GBP',
 'deadline': '2015-10-01',
 'goal': 6500.0,
 'launched': '2015-08-31 02:11:11',
 'pledged': 999.0,
 'state': 'failed',
 'backers': 12.0,
 'country': 'GB',
 'usd pledged': 1537.65168912,
 'usd_pledged_real': 1515.1974761876}

## Question 2

In [21]:
# list des projet qui ont eu plus de donnation que ce quiétait attenndu
cur = collection_ks.find({"$where" : "this.pledged >= this.goal"})
print("Il y a {} qui ont atteint leur but".format(len(list(cur))))

Il y a 53558 qui ont atteint leur but


## Question 3

In [22]:
#compter le nombre de projet par categori de projet
cur = collection_ks.aggregate([{"$group" : {"_id" : "$category", "numberPorjectByCategories" : {"$sum" : 1}}}])
list(cur)

[{'_id': 'Printing', 'numberPorjectByCategories': 83},
 {'_id': 'Mobile Games', 'numberPorjectByCategories': 650},
 {'_id': 'Robots', 'numberPorjectByCategories': 237},
 {'_id': 'Interactive Design', 'numberPorjectByCategories': 146},
 {'_id': 'Jazz', 'numberPorjectByCategories': 733},
 {'_id': 'Faith', 'numberPorjectByCategories': 439},
 {'_id': 'Documentary', 'numberPorjectByCategories': 6497},
 {'_id': 'Musical', 'numberPorjectByCategories': 367},
 {'_id': 'Performance Art', 'numberPorjectByCategories': 855},
 {'_id': 'Animation', 'numberPorjectByCategories': 1017},
 {'_id': 'Photo', 'numberPorjectByCategories': 58},
 {'_id': 'Glass', 'numberPorjectByCategories': 51},
 {'_id': 'Illustration', 'numberPorjectByCategories': 1263},
 {'_id': 'Space Exploration', 'numberPorjectByCategories': 137},
 {'_id': 'Live Games', 'numberPorjectByCategories': 394},
 {'_id': 'Technology', 'numberPorjectByCategories': 2689},
 {'_id': 'Graphic Novels', 'numberPorjectByCategories': 702},
 {'_id': 'Drama

## Question 4

In [23]:
#nombre de projet francais instacié avant 2016
cur = collection_ks.find({'country': 'FR', 'launched' : {'$lt': '2016-00-00 00:00:00'}})
len(list(cur))

330

In [24]:
#nombre de projet francais instacié avant 2016
cur = collection_ks.find({'country': 'FR', 'launched' : {'$lt': '2016-00-00 00:00:00'}}).aggregate({'$sum':1})
list(cur)

AttributeError: 'Cursor' object has no attribute 'aggregate'

## Question 5

In [25]:
# les projet americain qui ont demandé plus de 200000$
cur = collection_ks.find({'country' : 'US', 'usd_pledged_real' :{'$gt' : 200000}})
list(cur)

[{'_id': ObjectId('61b1dd1c90033986b96427d7'),
  'ID': 217543389,
  'name': 'The uKeg Pressurized Growler for Fresh Beer',
  'category': 'Drinks',
  'main_category': 'Food',
  'currency': 'USD',
  'deadline': '2014-12-08',
  'goal': 75000.0,
  'launched': '2014-10-15 06:34:48',
  'pledged': 1559525.68,
  'state': 'successful',
  'backers': 10293.0,
  'country': 'US',
  'usd pledged': 1559525.68,
  'usd_pledged_real': 1559525.68},
 {'_id': ObjectId('61b1dd1c90033986b9642866'),
  'ID': 909248984,
  'name': 'Redux COURG - Hybrid Watches with Missions to Tackle',
  'category': 'Product Design',
  'main_category': 'Design',
  'currency': 'USD',
  'deadline': '2015-08-20',
  'goal': 30000.0,
  'launched': '2015-07-21 19:01:41',
  'pledged': 692912.0,
  'state': 'successful',
  'backers': 2200.0,
  'country': 'US',
  'usd pledged': 692912.0,
  'usd_pledged_real': 692912.0},
 {'_id': ObjectId('61b1dd1c90033986b9642913'),
  'ID': 1688905333,
  'name': 'Legion Solar - A Better Way to Energy Inde

## Question 6 

Compter le nombre de projets ayant le mot sport dans le 'name'

In [26]:
#créationn d'un index textuel 
collection_ks.create_index([("name","text")])    

'name_text'

In [27]:
#Utilisation de l'index textuel pour cherhcer grasse a l'opérateur '$search' 
#la str 'sport' dans les 'name' des documents
cur = collection_ks.find({"$text" : {"$search":'Sport'}})
nb_project = len(list(cur))
print("Il y a {} qui possédent la str 'Sport' dans leur 'name'".format(nb_project))

Il y a 316 qui possédent la str 'Sport' dans leur 'name'
